# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

###### Цель исследования:
Построить наиболее качественную модель, которая выберет подходящий тариф для клиента оператора мобильной связи «Мегалайн».

###### Ход исследования:
1. Открыть и изучить предоставленные данные и подготовить их к исследованию при необходимости.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры.
4. Проверить качество модели на тестовой выборке.
5. Проверить модель на вменяемость.
6. Сделать вывод.

## Откройте и изучите файл

Импортируем библиотеки.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

Откроем файл с данными и изучим их.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Файл действительно содержит данные, заявленные в документах к нему.
На всякий случай проверим данные на пропуски и явные дубликаты, несмотря на то, что заявлено о их полной готовности к исследованию. Изменения типов данных не требуются.

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

Пропусков и дубликатов нет.

Вывод: данные изучены. в предобработке дополнительно не нуждаются. Таблица содержит все необходимые данные, которые и были заявлены в документах. Можно приступать к работе над основной задачей - построению модели по рекоммендации тарифа пользователям.

## Разбейте данные на выборки

1. Объявим две переменные - признаки (features) и целевой признак (target). Целевым признаком будет являться колонка 	is_ultra, остальные колонки таблицы - это features

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [7]:
print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


Итак данные разделены на признаки и целевой признак. Далее нужно разделить все данные на обучающую, валидационную и тестовую выборки. Данные должны быть разбиты в соотношении 3:1:1 соответственно.

2. Выделим обучающую выборку. Она должна составить 60% от всего массива данных. Для начала выделим его.

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345) 

Размер обучающей выборки получился такой:

In [9]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Молодец, что после разбиения данных на выборки смотришь на их размеры и размерности. Метод shape для этого - идеальный помощник. "Цифры" по выборкам показывают верно ли мы произвели "разделение" данных.

p.s. так (**псевдокод**): X_train.shape[0] – «покажет» количество строк в тренировочной выборке, а X_train.shape[1] - количество столбцов в ней же. Ну а X_train.shape – выведет размерность train'а в виде кортежа с 2мя значениями (первое число – количество строк, второе – столбцов).</div>

3. Оставшиеся 40% данных разделим пополам на валидационную и тестовую выборки.

In [18]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           )

<div class="alert alert-warning">
<b>Комментарий 👉</b>

Обрати внимание на ячейку, которую я сейчас комментирую – видишь слайдер («бегунок») снизу? Он показывает, что код не помещается в видимые границы ячейки (и чтобы его полностью «прочитать» - нужно использовать мышь. Код очень длинный.

Код очень длинный (а PEP8 определяет длину строки в 79 символов) - попробуй длинные - переносить, а между строками использовать пустые строки. Ну и желательно код комментировать, хотя бы коротко (что делаешь и каков будет результат - так твоим коллегам будет проще за твоей мыслью следить). Этот комментарий может относиться к нескольким ячейкам этой «тетрадки».

Хороший разработчик форматирует и комментирует свой код, тогда другие разработчики быстрее разберутся в программе. Это важно, ведь код чаще читают, чем пишут: напишет один, а прочтут — сотни или тысячи (если код хороший). 
«Пишите код так, как будто сопровождать его будет склонный к насилию психопат, который знает, где вы живете.» ©1991, John F. Woods, разработчик

---
Давай обсудим стиль кодирования (программирования)? Смотри что сейчас происходит в этой ячейке (**не всё применимо к твоему коду здесь, но я попытался собрать и представить здесь самые ценные советы из PEP8 по организации кода**) и что можно сделать по-другому, чуть лучше:

1. Комментарии к строке кода: они должны располагаться выше соответствующей строки кода (выше строки к которой относятся) и должны быть от этой строки отделены пустой строкой. Если же они расположены справа, то для того чтобы увидеть, что там написано приходится пользоваться скроллиногом. Надо добиться того, чтобы ячейку не приходилось прокручивать, т.к. это не добавляет удобства ни при отладке кода, ни для понимания что в этой ячейке "происходит". Как альтернатива – комментарий может размещаться выше, в Markdown-ячейке.
    
2. Бывает, что встречаются очень длинные строки кода, которые также уходят за правый край ячейки. Такие строки должны разбиваться разделителем (вот таким: \\). Это best practice, требование PEP8, а именно: PEP8 определяет длину строки в 79 символов.
    
3. **В идеале**: одна ячейка - одна строка кода. Почему так? Да как минимум из соображения понятности: будучи выполненной, под этой ячейкой должен отображаться результат трансформации данных. Ты или твой коллега должны видеть, как поменялись данные после выполнения ячейки с кодом. Следующая ячейка - ещё строка кода и соответствующим output'ом. Будешь делать так - коллеги будут тебе благодарны за то, что легко будут следовать за твоей мыслью. И отладку это упрощает. Ну и как противоположность - можно в одну ячейку "положить" результат 3-х последовательных groupby + ещё чего-нибудь. Разобраться с ходу "что происходит" будет очень трудно, придётся всё равно код разносить по ячейкам и смотреть последовательно что происходит. Но ведь логично это делать удобным способом сразу, ведь так?
    
4. Неиспользуемые, закомментированные строки выше вынеси в другую ячейку, отдельную, выше. Можешь для удобства своего прокомментировать (на память). И оставь там это код, так, чтобы он не смешивался с рабочим кодом.

p.s. возможно и в следующих ячейках твоей «тетрадки» будет код, который просмотреть можно только используя мышку, но я этот момент более комментировать не буду – просто обращай внимание на слайдер под ячейкой с кодом.
</div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Перенесла строчки кода, теперь не нужно скроллить. Спасибо.</font>
</div>

Размер валидационной выборки:

In [11]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


Размер тестовой выборки:

In [12]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


Вывод: данные разделены на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

<div class="alert alert-warning">
<b>Комментарий 👉</b>

Один момент "на будущее" (сейчас забегаю чуть вперед, но будущем это будет важно) который я хотел озвучить перед переходом к работе с моделями:

В наших данных есть числовые признаки (причём резко отличаюшиеся по значению модуля ... есть значения около 1 (маленькие, например *messages*) и десятки тысяч (большие, например *mb_used*). Линейные модели очень не любят подобные "разбеги" (вернее любят ))) потому что очень легко видят подобную разницу и придают бОльшим значениям бОльший вес в модели). После разделения данных на выборки, именно после разделения(!) следует отмасштабировать числовые столбцы. Используй StandardScaler() для этого.

Для чего нужно масштабировать признаки перед подачей в линейную модель машинного обучения (в линейную регрессию, в логистическую регрессию)? Нам нужно минимизировать функцию потерь методом градиентного спуска. В случае разных масштабов признаков (например, год рождения и количество детей) её линии уровня будут иметь вид вытянутых эллипсов. Тогда вектор антиградиента и направление от текущей точки к минимуму функции потерь могут не совпадать, мы можем уйти далеко и не в ту сторону, и, в зависимости от шага градиентного спуска, либо придётся сделать больше итераций, либо вообще не будет сходимости. Если же признаки отмасштабированы, то линии уровня похожи на окружности. И проблема несовпадения антиградиента и направления к минимуму не так выражена.
    
Таким образом, масштабирование признаков перед подачей их в логистическую регрессию имеет несколько причин и преимуществ:

- **Повышение стабильности и скорости сходимости**: Масштабирование признаков может помочь повысить стабильность и скорость сходимости алгоритма оптимизации, который используется при обучении логистической регрессии. Без масштабирования, признаки с большими значениями могут преобладать над признаками с меньшими значениями, что может замедлить сходимость алгоритма. Масштабирование позволяет привести все признаки к более сопоставимым диапазонам значений и улучшить процесс оптимизации.

- **Предотвращение проблем с градиентным спуском**: Градиентный спуск, используемый для оптимизации параметров логистической регрессии, может быть чувствителен к различным масштабам признаков. Если признаки имеют большие значения, градиенты могут быть слишком большими, что может привести к проблемам с численной стабильностью и затуханию градиентов. Масштабирование признаков помогает справиться с этими проблемами, нормализуя их значения.

- **Улучшение интерпретируемости**: Масштабирование признаков также может улучшить интерпретируемость модели. Когда признаки имеют разные диапазоны значений, сложнее понять, какой признак оказывает большее влияние на результаты модели. Масштабирование признаков позволяет сравнивать их вклады более непосредственно и делать более точные выводы о важности каждого признака.

В целом, хорошей практикой является проведение экспериментов с масштабированием и без него, и сравнение результатов моделей на валидационной выборке или с помощью кросс-валидации. Это поможет определить, как масштабирование признаков влияет на производительность и результаты модели в нашей конкретной задаче (но если стоит вопрос: масштабировать или не масштабировать, то ИМХО ответ однозначен: ДА, МАСШТАБИРОВАТЬ).

Ну и чуть-чуть теории (знаю, что в Тренажере этот момент вам пока не озвучивали): для масштабирования данных в линейной регрессии можно использовать различные методы, например стандартизацию (Standardization) и нормализацию (Normalization). Рассмотрим каждый из них с примерами кода (естественно используем наш scikit-learn).

**Стандартизация (Standardization):**
При стандартизации данные приводятся к стандартному нормальному распределению со средним значением 0 и стандартным отклонением 1. Это делается путем вычитания среднего значения и деления на стандартное отклонение каждого признака.
    
    from sklearn.preprocessing import StandardScaler

    # Создание объекта для стандартизации данных
    scaler = StandardScaler()

    # Масштабирование обучающего набора данных
    X_train_scaled = scaler.fit_transform(X_train)

    # Масштабирование тестового набора данных
    X_test_scaled = scaler.transform(X_test)
    
**Нормализация (Normalization):**
При нормализации данные приводятся к интервалу от 0 до 1 путем масштабирования каждого признака по его минимальному и максимальному значению.    
    
    from sklearn.preprocessing import MinMaxScaler

    # Создание объекта для нормализации данных
    scaler = MinMaxScaler()

    # Масштабирование обучающего набора данных
    X_train_scaled = scaler.fit_transform(X_train)

    # Масштабирование тестового набора данных
    X_test_scaled = scaler.transform(X_test)    
    
Обрати внимание, что данные масштабируются отдельно для каждого признака на основе значений в обучающем наборе данных. Затем трансформированные значения применяются как для обучающего, так и для тестового наборов данных <- ЭТО ОЧЕНЬ ВАЖНЫЙ МОМЕНТ (и здесь часто ошибки встречаются)!!!

Важно помнить, что масштабирование данных применяется только к признакам, а целевую переменную (выходные данные) необходимо оставить без изменений.

Выбор между стандартизацией и нормализацией зависит от конкретной задачи и характеристик данных. Рекомендуется экспериментировать с обоими методами и выбрать наиболее подходящий для конкретного случая (но по опыту скажу, что обычно ограничиваются стандартизацией).    
</div></div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Спасибо!</font>
</div>

<div class="alert alert-warning">
<b>Комментарий 👉</b>

Отлично, 5 шагов (EDA, обработка аномалий и пропусков, разделение данных на выборки, масштабирование/нормировку и кодирование признаков), входящие в типичный пайплайн подготовки данных **к этому моменту МОГУТ быть выполнены** ...

Я тебе совет дам: создай шаблон кода (или функцию/функции), которые будут работать с момента загрузки данных и до получения выборок, готовых к передаче в модели МО.
    
Зачем такой шаблон нужен? Чтобы не ломать всякий раз голову, не вспоминать и писать руками один и тот же код (да ещё и с возможностью совершить "глупую" ошибку). В шаблоне должны быть (как минимум) следующие рутинные процедуры, которые встречаются ну абсолютно в каждом МО-проекте:
    
1. разбиение данных на выборки,

2. масштабирование/нормировка.    

3. обработка категорий,
    
Затем ты этот шаблон будешь вставлять в каждый новый проект, чуть модифицировать его (имена переменных, названия столбцов) и гарантированно получать работающий и безошибочный код.
    
А ресурсы головы ))) лучше тратить на подбор гиперпараметров и генерацию новых фичей - вот здесь находится то, что способно выделить твои модели, поднять их качество.    
</div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Спасибо за совет.</font>
</div>

<div class="alert alert-success">
<b>КОММЕНТАРИЙ V2</b> 	

Вот пример ЧЕРНОВИКА шаблона пайплайна обработки данных для машинного обучения, включающий деление данных на выборки, кодирование категориальных переменных и масштабирование числовых переменных:
    
    from sklearn.model_selection import train_test_split
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler, OneHotEncoder

    # 1. Загрузка и подготовка данных
    # Загрузка данных
    data = ...

    # Разделение на признаки (X) и целевую переменную (y)
    X = data.drop(columns=['target'])
    y = data['target']

    # 2. Разделение данных на обучающую, валидационную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # 3. Создание пайплайна обработки данных
    # Предобработка числовых переменных
    numeric_features = [...]  # список числовых переменных

    numeric_transformer = StandardScaler()

    # Предобработка категориальных переменных
    categorical_features = [...]  # список категориальных переменных

    categorical_transformer = OneHotEncoder()

    # Комбинированный препроцессор для числовых и категориальных переменных
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # 4. Создание пайплайна для обучения модели
    from sklearn.linear_model import LinearRegression

    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', LinearRegression())])

    # 5. Обучение модели
    model.fit(X_train, y_train)

    # 6. Оценка модели
    train_score = model.score(X_train, y_train)
    val_score = model.score(X_val, y_val)
    test_score = model.score(X_test, y_test)

    # Вывод результатов
    print('Train Score:', train_score)
    print('Validation Score:', val_score)
    print('Test Score:', test_score)
    
    
В данном примере:

Шаг 1: Загружаются и подготавливаются данные.
    
Шаг 2: Данные разделяются на обучающую, валидационную и тестовую выборки с помощью функции train_test_split().
    
Шаг 3: Создается пайплайн обработки данных с использованием ColumnTransformer, который применяет соответствующие преобразования к числовым и категориальным переменным.
    
Шаг 4: Создается пайплайн для обучения модели, включающий предобработку данных и модель.
    
Шаг 5: Модель обучается на обучающих данных с помощью метода fit().
    
Шаг 6: Оценивается производительность модели на обучающей, валидационной и тестовой выборках с помощью метода score().    
</div>

## Исследуйте модели

Так как задача на классификацию (задача модели выбрать для аббонента один из двух возможных тарифов - Смарт(0) или Ультра(1)), для ее решения будем использовать соответствующие модели:
1. Решающее дерево(DecisionTreeClassifier)
2. Случайный лес(RandomForestClassifier)
3. Логистическая регрессия(LogisticRegression)

Для проверки качества моделей будем использовать метрику accuracy. Чем больше получится данный показатель, тем более качественна модель.

### Модель 1. Решающее дерево(DecisionTreeClassifier)

Напишем код, который позволит нам выбрать наиболее качественную из возможных вариантов модели Решающего дерева. Для этого будем перебирать гиперпараметр max_depth от 1 до 10

In [13]:
best_model_tree = None
best_depth_tree = 0
best_result_tree = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_tree.fit(features_train, target_train)
    predictions_valid_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_valid_tree)
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_depth_tree = depth
        best_result_tree = result_tree
        
print("Accuracy лучшей модели:", best_result_tree)
print ("Глубина лучшей модели:", best_depth_tree)

Accuracy лучшей модели: 0.7853810264385692
Глубина лучшей модели: 3


<div class="alert alert-warning">
<b>Комментарий 👉</b>

Рекомендую перебрать в цикле 2-3 гиперпараметра. Одного, как для "дерева", так и "леса" мало - точность наверняка будет низкой. В итоге получишь сразу два плюса: потренируешся работать с несколькими гиперпараметрами одновременно и получишь более высокую точность моделей!
</div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Например, так как я написала ниже? Добавить в цикл перебор гиперпараметра min_samples_split. Если так, то это действительно влияет на качество модели. Качество модели решающего дерева увеличилось с 0.785 до 0.819 если ограничить количество узлов, которые создает модель. </font>
</div>

<div class="alert alert-success">
<b>КОММЕНТАРИЙ V2</b> 	

Да, всё верно, как ниже, двойной цикл.
</div>


In [22]:
best_model_tree = None
best_depth_tree = 0
best_split_tree = 0
best_result_tree = 0
for depth in range(1, 11):
    for split in range(2, 5):
        model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth, min_samples_split = split)
        model_tree.fit(features_train, target_train)
        predictions_valid_tree = model_tree.predict(features_valid)
        result_tree = accuracy_score(target_valid, predictions_valid_tree)
        if result_tree > best_result_tree:
            best_model_tree = model_tree
            best_depth_tree = depth
            best_split_tree = split
            best_result_tree = result_tree
        
print("Accuracy лучшей модели:", best_result_tree)
print ("Глубина лучшей модели:", best_depth_tree)
print ("Минимальное количество примеров для разделения лучшей модели:", best_split_tree)

Accuracy лучшей модели: 0.8193146417445483
Глубина лучшей модели: 7
Vинимальное количество примеров для разделения лучшей модели: 2


Вывод: Качество (accuracy) лучшей модели Решающего дерева из исследуемых десяти равно 0.7853810264385692 при гиперпараметре max_depth равном 3.

### Модель 2. Случайный лес(RandomForestClassifier)

Аналогично исследуем модель Случайный лес. В данном случае нужно будет найти лучший параметр accuracy, перебирая не только глубину, но и количество деревьев. Для исследования возьмем глубину от 1 до 10, и количество деревьев от 1 до 20.

In [14]:
best_model_forest = None
best_est_forest = 0
best_depth_forest = 0
best_result_forest = 0
for est in range(1, 21):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) 
        model_forest.fit(features_train, target_train)
        predictions_valid_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(target_valid, predictions_valid_forest)
        if result_forest > best_result_forest:
            best_model_forest = model_forest
            best_est_forest = est
            best_depth_forest = depth
            best_result_forest = result_forest
        
print("Accuracy лучшей модели:", best_result_forest)
print("Количество деревьев лучшей модели:", best_est_forest)
print ("Глубина лучшей модели:", best_depth_forest)

Accuracy лучшей модели: 0.8040435458786936
Количество деревьев лучшей модели: 12
Глубина лучшей модели: 6


<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Ниже добавила в код модели леса тот же гиперпараметр min_samples_split. Качество так же возрасло. Единственное, такой код долго выполняется.</font>
</div>

<div class="alert alert-success">
<b>КОММЕНТАРИЙ V2</b> 	

Ну да, с увеличением количества подбираемых гиперпараметров МОЖЕТ(!) улучшаться и метрика, но однозначно будет увеличиваться и время расчета.
</div>


In [23]:
best_model_forest = None
best_est_forest = 0
best_depth_forest = 0
best_split_forest = 0
best_result_forest = 0
for est in range(1, 21):
    for depth in range(1, 11):
        for split in range(2, 5):
            model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth, 
                                                  min_samples_split = split) 
            model_forest.fit(features_train, target_train)
            predictions_valid_forest = model_forest.predict(features_valid)
            result_forest = accuracy_score(target_valid, predictions_valid_forest)
            if result_forest > best_result_forest:
                best_model_forest = model_forest
                best_est_forest = est
                best_depth_forest = depth
                best_split_forest = split
                best_result_forest = result_forest
        
print("Accuracy лучшей модели:", best_result_forest)
print("Количество деревьев лучшей модели:", best_est_forest)
print ("Глубина лучшей модели:", best_depth_forest)
print ("Минимальное количество примеров для разделения лучшей модели:", best_split_forest)

Accuracy лучшей модели: 0.8348909657320872
Количество деревьев лучшей модели: 6
Глубина лучшей модели: 8
Минимальное количество примеров для разделения лучшей модели: 3


Вывод: Качество (accuracy) лучшей модели Случайного леса из исследуемых равно 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.

### Модель 3. Логистическая регрессия(LogisticRegression)

Так же исследуем третью модель - Логистическую регрессию. Добавим гиперпараметры: solver='lbfgs' и max_iter=1000

In [15]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regression.fit(features_train, target_train)
predictions_valid_regression = model_regression.predict(features_valid)
result_regression = accuracy_score(target_valid, predictions_valid_regression)

print("Accuracy LogisticRegression:", result_regression)

Accuracy LogisticRegression: 0.7107309486780715


Вывод: Accuracy модели LogisticRegression равно 0.7107309486780715

### Вывод. 

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier).Ее показатель accuracy составил 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.

На втором месте по качеству оказалась модель Решающее дерево(DecisionTreeClassifier) с показателем accuracy 0.7853810264385692.

Хуже всех по качеству оказалась модель Логистическая регрессия(LogisticRegression) с показателем accuracy 0.7107309486780715.

<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Здесь отлично: подобрали для всех наших моделей лучшие гиперпараметры (в данном случае - максимизирующие метрику accuracy_score). Также здесь мы ещё и определили САМУЮ лучшую модель. На валидации ею оказалась модель "случайного леса". 

После того, как гиперпараметры на валидации подобраны - тестируем модели на тестовых данных. По результатам тестирования на тесте (сорри за тавталогию) выбираем модель, которую сможем передать в продакшн. 
</div>

## Проверьте модель на тестовой выборке

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier). Проверим данную модель на тестовой выборке.

In [16]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators = 12, max_depth = 6)
model_forest.fit(features_train, target_train)
predictions_test_forest = model_forest.predict(features_test)
accuracy_test_forest = accuracy_score(target_test, predictions_test_forest)

print("Accuracy модели случайного леса на тестовой выборке:", accuracy_test_forest)

Accuracy модели случайного леса на тестовой выборке: 0.7947122861586314


<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Здесь отлично: проверили "качество" нашей лучшей модели на тестовых данных и поняли что можем запустить её в промышленную эксплуатацию!
</div>

### Вывод. 
Accuracy модели случайного леса на тестовой выборке составила 0.7947122861586314, что выше заданного обязательного параметра 0.75. Таким образом, делаем вывод, что выбранная модель работает успешно.

## (бонус) Проверьте модели на адекватность

С помощью DummyClassifier создадим простейшую модель, которая будет предсказывать наиболее часто встречающийся класс. Найдем показатель accuracy для этой модели и сравним его с показателем accuracy для модели случайного леса, который показал свою эффективность в тестовой выборке в предыдущем задании. Если они будут различаться, значит модель адекватна, и обучилась верно, она действительно делает предсказание на основе всех параметров, и это не случайные предсказания.

In [17]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train, target_train)
predictions_dummy = dummy_model.predict(features_test)
print("Accuracy модели DummyClassifier:", accuracy_score(target_test, predictions_dummy))

Accuracy модели DummyClassifier: 0.6842923794712286


<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Верно, здесь мы используем простейшую (dummy) модель: DummyClassifier (для нашей задачи бинарной классификации). DummyClassifier "предсказывает" наиболее часто встречающийся класс. Здесь мы получаем контрольную accuracy, чтобы сравнить её с результатом работы нашей самой лучшей модели. Ну и конечно, из общих соображений понятно, что наша лучшая модель должна "побить" DummyClassifier.
    
p.s. (на перспективу!): можно было бы построить Confusion Matrix, чтобы детально посмотреть где ошибается модель. Подробнее о Confusion Matrix здесь: https://neptune.ai/blog/evaluation-metrics-binary-classification     

p.s. ещё вариант - сравнить самый часто встречающийся класс в наших данных (это is_ultra == 0). Таких значений 2.229 в нашем датафрейме. Всего же значений в датафрейме 3.214. Значит самый часто встречающийся класс "занимает" 69% (2229 / 3214 == 0.693528313627878). Вот мы и получили контрольные данные для сравнительной оценки, построенной нами "лучшей" модели. Наша лучшая модель должна "побить" этот скор.
</div>


<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Спасибо за подсказки, изучу. Долго думала над этой задачей и изучала что такое DummyClassifier, до остальных способов просто не добралась) Теперь есть, что почитать и поразбираться. </font>
</div>

Вывод: Модель Случайного леса, которую мы опрелили как качественную и успешную, адекватна, т.к. показатель Accuracy нашей модели составил 0.7947122861586314, что выше показателя Accuracy простейшей модели DummyClassifier (0.6842923794712286). Это значит, что наша модель делает не случайные предсказания.
Кроме того, мы также можем сделать вывод, что каждая из построенных нами моделей в предыдущем задании, адекватна.

## Общий вывод

#### Этап 1. Изучение данных.
В ходе этого этапа был открыт файл и изучены данные. Они действительно соответствовали документации. Так же была проведена проверка на пропуски и дубликаты. Их обнаружено не было, предобработка данных не потребовалась.

#### Этап 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
Данные были разделены на три выборки в соотношении 3:1:1. 60% данных было отведено на обучающую выборку, 20% - на валидационную и еще 20% на тестовую.

#### Этап 3. Исследование качества разных моделей.
В ходе работы были исследованы 3 разных модели:
1. Решающее дерево(DecisionTreeClassifier)
2. Случайный лес(RandomForestClassifier)
3. Логистическая регрессия(LogisticRegression)
Для проверки качества моделей была использована метрика accuracy.

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier).Ее показатель accuracy составил 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.
На втором месте по качеству оказалась модель Решающее дерево(DecisionTreeClassifier) с показателем accuracy 0.7853810264385692.
Хуже всех по качеству оказалась модель Логистическая регрессия(LogisticRegression) с показателем accuracy 0.7107309486780715.

#### Этап 4. Проверка качества модели на тестовой выборке.
На тестовой выборке была проверена модель Случайного леса(RandomForestClassifier), которая показала наиболее высокий показатель качества. Accuracy модели случайного леса на тестовой выборке составила 0.7947122861586314, что всего на 0.01 меньше, чем на валидационной. Модель рабочая.

#### Этап 5. Проверка модели на адекватность.
Была создана простейшая модель, которая предсказывает наиболее часто встречающийся класс. В результате сравнения качества простейшей модели и выбранной нами эффективной модели, было выявлено, что показатель Accuracy нашей модели составил 0.7947122861586314, это выше показателя Accuracy простейшей модели DummyClassifier (0.6842923794712286). Это значит, что наша модель делает не случайные предсказания.

<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Всё отлично, результат достигнут.
    
Один совет (также, на будущее): как попытаться улучшить полученный результат, с минимум усилий? Ответ: мы использовали train для обучения модели, а valid - для поиска лучших значений гиперпараметров. Лучшие параметры нашли. Так почему бы теперь наши модели с выбранными гиперпараметрами не обучить на *общей* (тренировочной + валидационной) выборке (pd.concat() можно использовать для объединения). Чем больше данных, тем *лучше* модели смогут обучиться (надо проверять!). И вот теперь эту дообученную модель мы уже финально проверим на тестовой выборке (test).

Но следует учесть вот какой момент: нужно быть аккуратным с подобным «улучшением», если мы кодируем или масштабируем наши выборки. Например, в следующем проекте мы обучаемся на train’е, а затем делаем transform на валидации и тесте. Если после этого объединить трейн и валид, то это будет не совсем верно.

p.s. а вообще, достижение высокого скора не такая простая задача. Много моментов может влиять (подбор гиперпараметров - всего лишь один из них). В этой конкретно задаче мне кажется, что данных маловато, а также, возможно и фичей (столбцов в наших данных). Из-за этого модели не могут точно "настроиться на данные".
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Елизавета, у тебя хорошая работа, все четко, осмысленно. Выводы присутствуют, с комментированием кода тоже никаких проблем нет.

Я оставил ряд советов на будущее (на результат этого проекта они не влияют). 

Я отправляю тебе проект ещё раз лишь для того, чтобы удостовериться что у тебя нет вопросов ко мне. Если их нет - пошли мне тетрадку ещё раз, и я тут же, без слов приму твою работу! 
</div>